# Basic Examples with Different Protocols

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

## Install Seldon Analytics

In [ ]:
!helm install seldon-core-analytics ../../../helm-charts/seldon-core-analytics -n seldon-system --wait

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
import json
import time

## Custom Metrics with a REST model

In [ ]:
!pygmentize model_rest.yaml

In [ ]:
!kubectl create -f model_rest.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-model \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
responseRaw=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}'    -X POST http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/predictions    -H "Content-Type: application/json"

In [ ]:
response=json.loads(responseRaw[0])
print(response)
assert(len(response["meta"]["metrics"]) == 3)

In [ ]:
print("Waiting so metrics can be scraped")
time.sleep(10)

In [ ]:
%%writefile get-metrics.sh

kubectl run --quiet=true -it --rm curlmetrics --image=tutum/curl --restart=Never -- \
    curl -s seldon-core-analytics-prometheus-seldon.seldon-system/api/v1/query?query=mycounter_total

In [ ]:
responseRaw =! bash get-metrics.sh

In [ ]:
responseRaw[0]

In [ ]:
response=json.loads(responseRaw[0])
print(response)
assert(response['data']["result"][0]["metric"]["__name__"]=='mycounter_total')

In [ ]:
!kubectl delete -f model_rest.yaml

## Custom Metrics with a GRPC model

In [ ]:
!pygmentize model_grpc.yaml

In [ ]:
!kubectl create -f model_grpc.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-model \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
responseRaw=!cd ../../../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0]]}}' \
         -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict

In [ ]:
response=json.loads("".join(responseRaw))
print(response)
assert(len(response["meta"]["metrics"]) == 3)

In [ ]:
print("Waiting so metrics can be scraped")
time.sleep(10)

In [ ]:
responseRaw =! bash get-metrics.sh

In [ ]:
response=json.loads(responseRaw[0])
print(response)
assert(response['data']["result"][0]["metric"]["__name__"]=='mycounter_total')
assert(response['data']["result"][0]["metric"]["image_name"]=='seldonio/model-with-metrics_grpc')

In [ ]:
!kubectl delete -f model_grpc.yaml

In [ ]:
!helm delete seldon-core-analytics --namespace seldon-system